In [1]:
from torch_geometric.data import HeteroData 
import torch 
import networkx as nx

data = HeteroData.from_dict(torch.load('HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'))

/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def top_k_mask(scores, indices, top_k ):
    # Make sure we are using the GPU
    scores = scores.cuda()
    indices = indices.cuda()
    
    # Create an empty mask with the same shape as scores
    mask = torch.zeros_like(scores, dtype=torch.bool)
    # Get the unique indices and their counts
    unique_indices, counts = torch.unique(indices, return_counts=True)

    # Indices where count > top_k
    large_indices = unique_indices[counts > top_k]

    # Set mask for indices where count <= top_k
    mask[~torch.isin(indices,large_indices)] = True
    # For indices where count > 50, we only keep top 50 scores
    for idx in tqdm(large_indices):
        idx_mask = (indices == idx)
        values, idxs = scores[idx_mask].topk(top_k)
        a = mask[idx_mask]
        a[idxs] = True
        mask[idx_mask] = a
        
    return mask.cpu()

top_k =50
import os
if True:
    print('for skill job edges keep top k edges per job, k is ',top_k)
    e = ('skills', 'job_skill', 'jobs')
    rev_e = (e[2],'rev_'+e[1],e[0])
    cache_dir = 'cache'
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    
    mask_path = os.path.join(cache_dir, f'mask{top_k}.pt') 
    
    if os.path.isfile(mask_path):
        mask = torch.load(mask_path)
    else:
        mask = top_k_mask(data[e].edge_attr.squeeze(1), data[e].edge_index[1,:], top_k)
        torch.save(mask, mask_path) 
    
    data[e].edge_attr = data[e].edge_attr[mask]
    data[rev_e].edge_attr = data[rev_e].edge_attr[mask]
    data[e].edge_index = data[e].edge_index[:,mask]
    data[rev_e].edge_index = data[rev_e].edge_index[:,mask]
    print('keep',torch.sum(mask), 'of total',mask.shape[0])

for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


In [3]:
def preprocess(data):
       
        for edge_type in data.edge_types:
            del data[edge_type].edge_attr 

        # delete all keys for every node type except 'x' (e.g. description and title)
        for node_type in data.node_types:
            keys = list(data[node_type].keys())
            for key in keys:
                if key != 'x':
                    del data[node_type][key]
        return data
    
data = preprocess(data)

In [4]:
# gc collect
import gc
gc.collect()


26

In [5]:
homogeneous_data = data.to_homogeneous()

: 

In [ ]:
nx_graph = data.to_networkx()

AttributeError: 'HeteroData' has no attribute 'to_networkx'